In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#matplotlib inline

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

import sys
sys.path.append('./src')

from data_queries import  getClusterInfo, getGAIAKnownMembers
from coeval import coeval
from gaiastars import gaiastars as gs,gaiadr2xdr3

import astropy.units as u
from astropy.coordinates import SkyCoord

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [3]:

known_cluster_members, cluster_names = getGAIAKnownMembers()
print(cluster_names)

# just deal with Pleiades and alphaPer for now
cluster_names = ['Pleiades', 'alphaPer']
xmatches = {}
cluster_members={}
#for cl in cluster_names:
for cl in cluster_names:
    known_members_dr2 = list(known_cluster_members.query('Cluster == @cl').index)
    xmatches[cl] = gaiadr2xdr3(known_members_dr2)
    cluster_members[cl]  = gs(name = cl, description=f'{cl} sources from Table 1a records from Gaia archive')
    cluster_members[cl].from_source_idlist(list(xmatches[cl].dr3_source_id),schema='gaiaedr3', query_type='sync')

#construct a dict mapping cluster name in Table1a to its name in Simabad
cluster_info = getClusterInfo()

cluster_names = ['Pleiades', 'alphaPer']
search_results = {}

from gaiastars import from_pickle
import time
import pickle

for cl in cluster_names:
    search_results[cl] = from_pickle(f'./data/search_results_{cl}.pkl')

#exclude the known members returned from the search
for cl in cluster_names:
    merged_fs = search_results[cl].merge(cluster_members[cl])
    print(f'------ {cl} -------')
    print(merged_fs.objs.which.value_counts())
    fs = merged_fs.query('which == \'{} cone search\''.format(cl))
    fs.name = 'Search Results, Known Members excluded'
    search_results[cl] = fs



['Hyades' 'ComaBer' 'Pleiades' 'Praesepe' 'alphaPer' 'IC2391' 'IC2602'
 'Blanco1' 'NGC2451']
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
------ Pleiades -------
Pleiades cone search    107919
both                      1225
Pleiades                   101
Name: which, dtype: int64
------ alphaPer -------
alphaPer cone search    291664
both                       699
alphaPer                    41
Name: which, dtype: int64


In [4]:
import pickle
coeval_results = {}
travel_times = {}

for cl in cluster_names:
    with open(f'./data/coeval_{cl}.pkl', 'rb') as pkl:
        coeval_results[cl]=pickle.load(pkl)
    with open(f'./data/travel_time_{cl}.pkl', 'rb') as pkl:
        travel_times[cl] = pickle.load(pkl)
    search_results[cl].travel_time(cluster_info.loc[cl]['coords'],inplace=True)
    search_results[cl].objs = pd.concat([search_results[cl].objs, coeval_results[cl], travel_times[cl]], axis=1)

In [5]:
cl = 'alphaPer'
cols = ['min_sep', 'lookback_time','rv','radial_velocity']
search_results[cl].objs.query('min_sep<=20 and lookback_time >= 70e6 and lookback_time <= 130e6')[cols]

,min_sep,lookback_time,rv,radial_velocity
source_id,,,,
115579773255527552,10.193571,1.166667e+08,6.306306,NaN
120009327646718848,9.196748,1.045045e+08,7.507508,3.048116
126989607278665600,6.944174,7.432432e+07,5.105105,NaN
127023004944384896,18.092147,8.243243e+07,4.304304,NaN
129460386001757824,17.141306,8.873874e+07,3.703704,NaN
161669475330756736,17.577989,7.207207e+07,11.311311,NaN
164646677940221568,8.546298,1.207207e+08,10.510511,NaN
168744115397150592,4.370380,1.099099e+08,7.307307,NaN
172510114519532544,11.403687,7.792793e+07,9.309309,9.461825


In [7]:
len(search_results[cl].objs.query('min_sep<=20 and lookback_time >= 70e6 and lookback_time <= 130e6')[cols])

39

In [6]:
cl = 'alphaPer'
cols = ['min_sep', 'lookback_time','travel_time','radial_velocity','rv']
search_results[cl].objs.query('travel_time >= 70e06')[cols]

,min_sep,lookback_time,travel_time,radial_velocity,rv
source_id,,,,,
68387630383786624,184.038602,5.000000e+07,7.683338e+07,NaN,9.909910
69933170071162624,267.038843,5.000000e+07,1.034778e+08,NaN,8.908909
70068650520031104,21.296406,1.369369e+08,1.009575e+08,NaN,9.509510
70247213783713280,342.888074,5.000000e+07,1.983844e+08,NaN,8.508509
70247218079697280,316.026876,5.000000e+07,9.400847e+07,NaN,8.508509
...,...,...,...,...,...
541402347311747200,675.698586,5.000000e+07,1.386406e+08,NaN,-6.906907
542416509349161728,840.081068,5.000000e+07,1.286077e+08,NaN,-7.307307
545017988220286720,413.893077,5.000000e+07,7.212858e+07,NaN,-9.109109
